#### GroupBy Mechanics

In [13]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
print(df, '\n')
grouped = df['data1'].groupby(df['key1'])
print(grouped, '\n')
print(grouped.mean(),'\n')
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
print(means, '\n')
print(means.unstack(), '\n')

#group key可以是任何合适长度的arrays，在这里是series
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
print(df['data1'].groupby([states, years]).mean(), '\n')
#对于dataframe可以把列名作为group keys   为什么key2没有自动加入？也没有被识别为数据
print(df.groupby('key1').mean(), '\n')
print(df.groupby(['key1','key2']).mean(), '\n')
#size()函数能得到每个group里的个数
print(df.groupby(['key1','key2']).size(), '\n')

  key1 key2     data1     data2
0    a  one -0.968666 -0.015209
1    a  two  0.787794 -0.762831
2    b  one -0.581772 -1.028280
3    b  two  1.905340  0.126133
4    a  one  0.374959  0.768326 


key1
a    0.064696
b    0.661784
Name: data1, dtype: float64 

key1  key2
a     one    -0.296853
      two     0.787794
b     one    -0.581772
      two     1.905340
Name: data1, dtype: float64 

key2       one       two
key1                    
a    -0.296853  0.787794
b    -0.581772  1.905340 

California  2005    0.787794
            2006   -0.581772
Ohio        2005    0.468337
            2006    0.374959
Name: data1, dtype: float64 

         data1     data2
key1                    
a     0.064696 -0.003238
b     0.661784 -0.451074 

              data1     data2
key1 key2                    
a    one  -0.296853  0.376558
     two   0.787794 -0.762831
b    one  -0.581772 -1.028280
     two   1.905340  0.126133 

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dt

In [18]:
print('####################Iterating Over Groups#########################')
for name, group in df.groupby('key1'):
    print(name)
    print(group)
print('\n\n')
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)
print('\n\n')
#用字典方式把group后的每一份分到一个key 和value里
pieces = dict(list(df.groupby('key1')))
print(pieces['b'] ,'\n')
print(df.dtypes)
#default里axis=0，可以设定axis=1对列操作
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

####################Iterating Over Groups#########################
a
  key1 key2     data1     data2
0    a  one -0.968666 -0.015209
1    a  two  0.787794 -0.762831
4    a  one  0.374959  0.768326
b
  key1 key2     data1     data2
2    b  one -0.581772 -1.028280
3    b  two  1.905340  0.126133



('a', 'one')
  key1 key2     data1     data2
0    a  one -0.968666 -0.015209
4    a  one  0.374959  0.768326
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.787794 -0.762831
('b', 'one')
  key1 key2     data1    data2
2    b  one -0.581772 -1.02828
('b', 'two')
  key1 key2    data1     data2
3    b  two  1.90534  0.126133



  key1 key2     data1     data2
2    b  one -0.581772 -1.028280
3    b  two  1.905340  0.126133 

key1      object
key2      object
data1    float64
data2    float64
dtype: object


In [20]:
print('####################Selecting a Column or Subset of Columns#########################')
df.groupby('key1')['data1']

####################Selecting a Column or Subset of Columns#########################


#### 2 Data Aggregation

In [26]:
print(df, '\n')
grouped = df.groupby('key1')
print(grouped)
print(grouped['data1'].quantile(0.9), '\n')
#使用自己的aggregation公式，可以aggregate一个array去aggregate或者agg
def peak_to_peak(arr):
    return arr.max() - arr.min()
print(grouped.agg(peak_to_peak), '\n')
print(grouped.describe())

  key1 key2     data1     data2
0    a  one -0.968666 -0.015209
1    a  two  0.787794 -0.762831
2    b  one -0.581772 -1.028280
3    b  two  1.905340  0.126133
4    a  one  0.374959  0.768326 

key1
a    0.705227
b    1.656629
Name: data1, dtype: float64 

         data1     data2
key1                    
a     1.756460  1.531157
b     2.487112  1.154413 

     data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.064696  0.918415 -0.968666 -0.296853  0.374959  0.581377   
b      2.0  0.661784  1.758654 -0.581772  0.040006  0.661784  1.283562   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.787794   3.0 -0.003238  0.765649 -0.7628

In [28]:
print('####################Column-Wise and Multiple Function Application#########################')
tips = pd.read_csv('../examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
print(tips[:6], '\n')
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
print(grouped_pct.agg('mean'),'\n')

####################Column-Wise and Multiple Function Application#########################
   total_bill   tip smoker  day    time  size   tip_pct
0       16.99  1.01     No  Sun  Dinner     2  0.059447
1       10.34  1.66     No  Sun  Dinner     3  0.160542
2       21.01  3.50     No  Sun  Dinner     3  0.166587
3       23.68  3.31     No  Sun  Dinner     2  0.139780
4       24.59  3.61     No  Sun  Dinner     4  0.146808
5       25.29  4.71     No  Sun  Dinner     4  0.186240 

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64 



#### 3 Apply: General split-apply-combine

In [32]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)
tips.groupby('smoker').apply(top)
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')
result = tips.groupby('smoker')['tip_pct'].describe()
print(result.unstack('smoker'),'\n')
print(tips.groupby('smoker', group_keys=False).apply(top), '\n')


frame = pd.DataFrame({'data1': np.random.randn(1000),
                       'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
print(quartiles[:10],'\n')


def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
print(grouped.apply(get_stats).unstack(), '\n')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64 

     total_bill   tip smoker   day    time  size   tip_pct
88        24.71  5.85     No  Thur   Lunch     2  0.236746
185       20.69  5.00     No   Sun  Dinner     5  0.241663
51        10.29  2.60     No   Sun  Dinner     2  0.252672
149        7.51  2.00     No  Thur   Lunch     2  0.266312
232       11.61  3.39     No   Sat  Dinner     2  0.291990
109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
67         3.07  1.00    Yes   Sat  Dinner     1 